In [94]:
!pip install numpy
!pip install scipy
import numpy as np
from scipy.interpolate import lagrange



# Part 1 (Interpolate function)

In [95]:

def interpolate(Y):
    length = len(Y)
    X = [x for x in range(length)]
    return lagrange(X, Y)

array = [1, 2, 5, 1, 2]
factor = 21
multipliedArray = [factor * i for i in array]
print(factor * interpolate(array))
print(interpolate(multipliedArray))

#There is a homomorphism between the column vectors and polynomials
#As a result, scalar multiplication can be done before or after the transformation
#End result is same


       4         3         2
18.38 x - 141.8 x + 317.6 x - 173.2 x + 21
       4         3         2
18.38 x - 141.8 x + 317.6 x - 173.2 x + 21


# Part 2 (R1CS->QAP over real numbers)

In [96]:
import numpy as np
import random

#Define the matrices
A = np.array([[0,0,3,0,0,0],
               [0,0,0,0,1,0],
               [0,0,1,0,0,0]])

B = np.array([[0,0,1,0,0,0],
               [0,0,0,1,0,0],
               [0,0,0,5,0,0]])

C = np.array([[0,0,0,0,1,0],
               [0,0,0,0,0,1],
               [-3,1,1,2,0,-1]])

#pick values for x and y
x = 100
y = 100

#this is our orignal formula
out = 3 * x * x * y + 5 * x * y - x- 2*y + 3 # the witness vector with the intermediate variables inside
v1 = 3*x*x
v2 = v1 * y
w = np.array([1, out, x, y, v1, v2])

result = C.dot(w) == np.multiply(A.dot(w),B.dot(w))
assert result.all(), "result contains an inequality"

In [97]:
#QAP By Hand

#R1CS: Cw = Aw ⊙ Bw 
"""
We have 3x6 matrices, and a 6x1 witness vector

After multiplying, we should get a 3x1 vector
"""

# LHS:
lhs = 0
for i in range(len(C[0])):
    lhs += interpolate(C[:, i]) * w[i] #Lets interpolate C to a polynomial!
print(lhs)

# RHS (first):
first = 0
for i in range(len(A[0])):
    first += interpolate(A[:, i]) * w[i] #Lets interpolate A to a polynomial!
print(first)

# RHS (second):
second = 0
for i in range(len(B[0])):
    second += interpolate(B[:, i]) * w[i] #Lets interpolate B to a polynomial!
print(second)

print(first * second)


t = np.poly1d([0, 1, 2], True)

top = (first*second - lhs)
print(t)
h = ((first*second - lhs) / t)[0]

assert(h * t == top) # clearly there was no remainder in the divison

rhs = first*second
i = random.randint(0, 421)

assert(lhs(i) + t(i)*h(i) == rhs(i))

print("lhs", lhs)

# Congratulations, you have proven that the R1CS holds for the witness w, 
# with an O(1) verification step (no need to check each row of matrix)

           2
-2.96e+06 x + 5.93e+06 x + 3e+04
           2
-2.98e+04 x + 5.95e+04 x + 300
     2
200 x - 200 x + 100
           4             3             2
-5.96e+06 x + 1.786e+07 x - 1.482e+07 x + 5.89e+06 x + 3e+04
   3     2
1 x - 3 x + 2 x
lhs            2
-2.96e+06 x + 5.93e+06 x + 3e+04


# Unanswered question
Q: Why exactly does u(x)*v(x) interpolate the same points as w(x)???

Why does A ⊙ B map to a(x) * b(x)?


### My notes on R1CS -> QAP

The R1CS was $ Aw ⊙ Bw == Cw $

We have interpolated each column of the matrices before multiplication
- $L(Aw) * L(Bw) == L(Cw)$
- $u(x)*v(x) == w(x)$

But remember: $Aw ⊙ Bw == Cw$ (So the vectors are the same, polynomials are different)

Problem: $u*v$ is degree 4, $w(x$ is degree 2 (so the polynomials aint equal)

What if we just changed the vectors in a way that polynomials change,

but vector value is unchanged? (this bit ensures that the r1cs is still satisfied)

Lets add the zero vector! (Interpolate first)

The resulting polynomial will still interpolate same points as w(x) and u(x)*v(x)

So, now we have $u(x)*v(x) == w(x) + b(x)$

(You can calculate $b(x)$ as $u(x)*v(x) - w(x)$)

The issue here- first assume the witness is wrong:
- That means that $Aw ⊙ Bw == Cw$ is false
- That means that $L(Cw)$ and $L(Aw) * L(Bw)$ would interpolate to different polynomials
- This means that $u(x)*v(x)$ - $w(x)$ does not give a poly that interpolates 0 vector
- But then we can still find $b(x)$ such that $u(x)*v(x)-w(x)-b(x) = 0$. false proof

So we need to ENFORCE that $b(x)$ actually interpolates the 0 vector
To do that, just give it the factors $x(x-1)(x-2)$

so $b(x) = (x-1)(x-2)(x-3) * t(x)$

Now, equation is $u(x)*v(x) == w(x) + (x)(x-1)(x-2)h(x)$

Note that since $u(x)*v(x) - w(x)$ also interpolates [0,0,0], it also has those 3 factors

So: $h(x) = \frac{u(x)*v(x) - w(x)}{[(x)(x-1)(x-2)]}$

# Part 3 (QAP over finite field)

In [127]:
import galois
p = 41
gf = galois.GF(p)


#R1CS: Aw ⊙ Bw = Cw

# Interpolation

def interpolate(col):
    X = gf(np.array([x for x in range(len(col))]))
    return galois.lagrange_poly(X, gf(col))


In [128]:
t = galois.Poly([1, 0], field = gf) * galois.Poly([1, p - 1], field = gf) * galois.Poly([1, p - 2], field = gf)

def compute_poly(matrix, w):
    poly = galois.Poly([0], field=gf)
    for i in range(len(matrix[0])):
        interpolated_poly = interpolate(gf(matrix[:, i] % p))
        poly += interpolated_poly * gf(w[i]%p)  # Interpolate and add in GF(79)
    return poly
print(t)
print(galois.Poly([w[0]], field=gf))

# Initialize LHS in GF(79)
rhs = compute_poly(C, w)
lhs = compute_poly(A, w) * compute_poly(B, w)

h = (lhs - rhs) // t
assert(h*t == lhs - rhs)

# QAP Assertion
r = random.randint(0, p-1)

assert(lhs(r) == rhs(r) + h(r)*t(r))
    



x^3 + 38x^2 + 2x
1
